# Creating a representative set of GitHub Repos

In [1]:
from github import Github, GithubException
import json
import pandas as pd
import configparser
from tqdm import tqdm

In [2]:
def get_access_token():
    """Read Github API access token from config file.

    Returns:
        str: Access Token
    """
    config = configparser.ConfigParser()
    config.read('../config.cfg')
    return config['ACCESS']['token']

In [3]:
g = Github(get_access_token())

## Southhampton ePrints

Considering just repos from within the last year in the Southhampton ePrints. Problem: quite few repos, plus missing huge repos with over 10000 commits.

In [4]:
with open("data/cleaned_urls_eprints.soton.ac.uk_2022-_github.com.json", "r") as f:
    repo_dict = json.load(f)

In [65]:
df_dict = {"pdf_link": [], "user_name": [], "repo_name": [], "stars": [], "watchers": [], "forks": [], "commits_no": [], "contributors_no": [], "size_kb": []}

In [66]:
for pdf_link, repos in tqdm(repo_dict.items()):
    for repo_data in repos:
        repo = g.get_repo(f"{repo_data['user']}/{repo_data['repo']}")
        df_dict["pdf_link"].append(pdf_link)
        df_dict["user_name"].append(repo_data['user'])
        df_dict["repo_name"].append(repo_data['repo'])
        df_dict["stars"].append(repo.get_stargazers().totalCount)
        df_dict["watchers"].append(repo.get_subscribers().totalCount)
        df_dict["forks"].append(repo.get_forks().totalCount)
        df_dict["commits_no"].append(repo.get_commits().totalCount)
        df_dict["contributors_no"].append(repo.get_contributors().totalCount)
        df_dict["size_kb"].append(repo.size)

100%|██████████| 17/17 [00:34<00:00,  2.04s/it]


In [67]:
df = pd.DataFrame(df_dict)

In [68]:
df

,pdf_link,user_name,repo_name,stars,watchers,forks,commits_no,contributors_no,size_kb
0,https://eprints.soton.ac.uk/457919/1/CLPsych_S...,stuartemiddleton,uos_clpsych,1,3,1,88,3,2985
1,https://eprints.soton.ac.uk/473266/1/2211.1562...,paboyle,Grid,132,30,84,6868,37,63648
2,https://eprints.soton.ac.uk/458195/1/SIGIR_202...,TGMclustering,TGMclustering,0,1,0,8,1,571
3,https://eprints.soton.ac.uk/457426/1/ACMFACCT_...,seatgeek,fuzzywuzzy,8848,268,883,384,60,336
4,https://eprints.soton.ac.uk/454952/1/2201.1170...,JAEarly,MILLI,10,2,1,3,1,24795
5,https://eprints.soton.ac.uk/454808/1/JPAL_AAMA...,ilkaza,JPAL-HA,1,2,0,17,1,1320
6,https://eprints.soton.ac.uk/457013/1/EmeCom_at...,olipinski,rl_werewolf,1,1,0,362,2,5524
7,https://eprints.soton.ac.uk/467378/1/Multiobje...,Miya-Liu,equitable-ridesharing,0,1,0,7,1,90114
8,https://eprints.soton.ac.uk/467378/2/4.pdf,Miya-Liu,equitable-ridesharing,0,1,0,7,1,90114
9,https://eprints.soton.ac.uk/468206/1/Camera_re...,AntoniaMarcu,Data-Modification,1,1,0,2,1,194


## Overall GitHub

"Sampling" from GitHub, taking stars as indicator.

In [5]:
def parse_samples(slice):
    samples_dict = {"user_name": [], "repo_name": [], "stars": [], "watchers": [], "forks": [], "commits_no": [], "contributors_no": [], "size_kb": []}
    for s in slice:
        samples_dict["user_name"].append(s.owner.login)
        samples_dict["repo_name"].append(s.name)
        samples_dict["stars"].append(s.get_stargazers().totalCount)
        samples_dict["watchers"].append(s.get_subscribers().totalCount)
        samples_dict["forks"].append(s.get_forks().totalCount)
        try:
            samples_dict["commits_no"].append(s.get_commits().totalCount)
        except GithubException:
            samples_dict["commits_no"].append(0)
        samples_dict["contributors_no"].append(s.get_contributors().totalCount)
        samples_dict["size_kb"].append(s.size)
    samples_df = pd.DataFrame(samples_dict)
    return samples_df

In [6]:
samples = {}
stars_intervals = ["<1", "1..100", "100..1000", "1000..10000", ">10000"]
for interval in tqdm(stars_intervals):
    result = g.search_repositories(f"stars:{interval} fork:false created:>2018-01-01")
    samples[interval] = parse_samples(result[:20])

100%|██████████| 5/5 [02:24<00:00, 29.00s/it]


In [7]:
samples["1000..10000"]

,user_name,repo_name,stars,watchers,forks,commits_no,contributors_no,size_kb
0,openai,DALL-E,9980,227,1812,3,1,7
1,lowlighter,metrics,9978,81,1250,2476,53,135373
2,charmbracelet,vhs,9977,26,150,515,20,39867
3,vipstone,faceai,9974,387,2396,150,4,40473
4,vaxilu,x-ui,9974,135,3723,80,6,15938
5,ffmpegwasm,ffmpeg.wasm,9965,102,515,386,34,9166
6,macrozheng,mall-swarm,9949,274,4624,448,1,53241
7,kautukkundan,Awesome-Profile-README-templates,9948,165,7015,344,236,482
8,pwxcoo,chinese-xinhua,9939,309,2306,15,3,36271
9,skywind3000,awesome-cheatsheets,9920,274,1851,334,33,396


ERROR! Only saved 20 entries!

In [8]:
df = pd.concat(samples.values())

In [9]:
len(df)

100

In [10]:
df.to_csv("data/representative_set.csv", index=False)